In [ ]:
import sys
sys.path.append("../input/tez-lib")
sys.path.append("../input/timmmaster")

In [ ]:
import tez
import albumentations
import pandas as pd
import numpy as np
import timm
from sklearn import metrics
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import math

import glob
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import models, transforms

In [ ]:
print("Pytorch version:",torch.__version__)
print("Torchvision version:", torchvision.__version__)

In [ ]:
#入力画像の前処理のクラス
class BaseTransform():
    def __init__(self,resize,mean,std):
        self.base_transform=transforms.Compose([
            transforms.Resize(resize), #短い辺の長さがresizeとなる
            transforms.CenterCrop(resize),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(), #Torchテンソルに変換
            transforms.Normalize(mean,std)
        ])
        
    def __call__(self,img):
        return self.base_transform(img)

In [ ]:
#seedを設定
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

set_seed(123)

In [ ]:
#入力画像の前処理をするクラス
#訓練時と推論時で処理が異なる

class ImageTransform():
    def __init__(self, resize, mean,std):
        self.data_transform={
            "train":transforms.Compose([
            transforms.RandomResizedCrop(
            resize,scale=(0.5,1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean,std)
        ]),
            "val": transforms.Compose([
                transforms.Resize(resize),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean,std)
            ])
        }
        
    def __call__(self,img,phase="train"):
        return self.data_transform[phase](img)   

In [ ]:
image_file_path="../input/petfinder-pawpularity-score/train/0007de18844b0dbbb5e1f607da0606e0.jpg"
img=Image.open(image_file_path)

#元の画像
plt.imshow(img)
plt.show()

#前処理済みの画像
resize=224
mean=(0.485, 0.456, 0.406)
std=(0.229, 0.224, 0.225)
transform=ImageTransform(resize, mean, std)
img_transformed=transform(img)

img_transformed=img_transformed.numpy().transpose((1,2,0))
img_transformed=np.clip(img_transformed,0,1)
plt.imshow(img_transformed)
plt.show()

In [ ]:
train_df=pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
train_df.head()

In [ ]:
train_df["Path"]=["../input/petfinder-pawpularity-score"+"/train/"+x+".jpg" for x in train_df["Id"]]

In [ ]:
train_df.head()

In [ ]:
dense_features = [
        'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
        'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']

In [ ]:
class PawpularDataset(Dataset):
    def __init__(self, df,dense_features,transform=None, phase="train"):
        self.df = df
        self.transform=transform
        self.phase=phase #train or val
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        image_path=self.df["Path"].iloc[index]
        img=Image.open(image_path)
        img_transformed=self.transform(img,self.phase) #torch.Size([3,224,224])
        
        dense_feature=self.df[dense_features].loc[index,:].values
        
        target=self.df["Pawpularity"].iloc[index]
    
        return img_transformed,dense_feature, target

In [ ]:
from sklearn.model_selection import train_test_split
traindf, valdf = train_test_split(train_df, test_size=0.2)

traindf=traindf.reset_index(drop=True)
valdf=valdf.reset_index(drop=True)

print(len(traindf))
print(len(valdf))

In [ ]:
train_dataset=PawpularDataset(traindf,dense_features,transform=ImageTransform(resize,mean,std),phase="train")
val_dataset=PawpularDataset(valdf,dense_features,transform=ImageTransform(resize,mean,std),phase="val")

#動作確認
index=0
print(train_dataset.__getitem__(index)[0].size())

In [ ]:
#DataLoaderを作成
batch_size=128

train_dataloader=DataLoader(
train_dataset,batch_size=batch_size,shuffle=True)

val_dataloader=DataLoader(
val_dataset,batch_size=batch_size,shuffle=False)

dataloaders_dict={"train": train_dataloader, "val": val_dataloader}

#動作確認
batch_iterator=iter(dataloaders_dict["train"])
inputs,_,targets=next(batch_iterator)
print(inputs.size())
print(targets)

In [ ]:
class VGG16Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.model=models.vgg16(pretrained=True)
        self.model.classifier[6]=nn.Linear(in_features=4096, out_features=2048)
        self.dropout1=nn.Dropout(0.5)
        self.dense1=nn.Linear(2048,256)
        self.dropout2=nn.Dropout(0.5)
        self.dense2=nn.Linear(256+12,32)
        self.dense3=nn.Linear(32,1)
        
    def forward(self,image,features,targets=None):
        x=self.model(image)
        x=self.dropout1(x)
        x=self.dense1(x)
        x=self.dropout2(x)
        x=torch.cat([x,features],dim=1)
        x=self.dense2(x)
        x=self.dense3(x)
        
        return x

In [ ]:
class SwinModel(nn.Module):
    def __init__(self,model_name):
        super().__init__()
        self.model=timm.create_model(model_name,pretrained=True,in_chans=3)
        self.model.head=nn.Linear(self.model.head.in_features,128)
        self.dropout1=nn.Dropout(0.5)
        self.dense1=nn.Linear(128,64)
        self.dropout2=nn.Dropout(0.5)
        self.dense2=nn.Linear(64+12,32)
        self.dense3=nn.Linear(32,1)
        
    def forward(self,image,features,targets=None):
        x=self.model(image)
        x=self.dropout1(x)
        x=self.dense1(x)
        x=self.dropout2(x)
        x=torch.cat([x,features],dim=1)
        x=self.dense2(x)
        x=self.dense3(x)
        
        return x

In [ ]:
class Resnet50Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.model=models.resnet50(pretrained=True)
        self.fc=nn.Linear(1000,128)
        self.dense1=nn.Linear(128,64)
        self.dropout1=nn.Dropout(0.5)
        self.dense2=nn.Linear(64+12,32)
        self.dropout2=nn.Dropout(0.5)
        self.dense3=nn.Linear(32,1)
        
    def forward(self,image,features,targets=None):
        x=self.model(image)
        x=self.fc(x)
        x=self.dense1(x)
        x=self.dropout1(x)
        x=torch.cat([x,features],dim=1)
        x=self.dense2(x)
        x=self.dropout2(x)
        x=self.dense3(x)
        
        return x

In [ ]:
def CustomLoss(output,target):
    return torch.sqrt(torch.mean((output-target)**2))

In [ ]:
def train_model(net,dataloaders_dict,criterion,optimizer,num_epochs):
    for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch+1, num_epochs))
        print("----------")
        
        for phase in ["train","val"]:
            if phase=="train":
                net.train()
            else:
                net.eval()
            
            epoch_loss=0.0
            
            #未学習時の検証性能を確かめるため、epoch0の訓練は省略
            if (epoch==0) and (phase=="train"):
                continue
        
            for images,features,targets in tqdm(dataloaders_dict[phase]):
                images=torch.tensor(images, dtype=torch.float)
                features=torch.tensor(features,dtype=torch.float)
                targets=torch.tensor(targets, dtype=torch.float)
                
                images=images.to(device)
                features=features.to(device)
                targets=targets.to(device)
           
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase=="train"):
                    outputs=net(images,features)
                    loss=criterion(outputs,targets)
                
                    #訓練時はbackpropagation
                    if phase=="train":
                        loss.backward()
                        optimizer.step()
                
                    #イテレーション結果の計算
                    #lossの合計を更新
                    epoch_loss+=loss.item()*inputs.size(0)

            #epochごとのlossを表示
            epoch_loss=epoch_loss/len(dataloaders_dict[phase].dataset)
            print("{}Loss: {:.4f}".format(phase,epoch_loss))
        
            if phase=="train":
                save_path="weights_{}.pth".format(epoch+1)
                torch.save(net.state_dict(),save_path)
            
            torch.cuda.empty_cache()

In [ ]:
#VGG16の学習
vgg16_model=VGG16Model()
VGG16_model.to(device)

num_epochs=10
vgg16_optimizer = torch.optim.AdamW(params=vgg16_model.parameters(),lr=0.001)
train_model(vgg16_model,dataloaders_dict,CustomLoss,vgg16_optimizer,num_epochs)

In [ ]:
#ResNet50の学習
resnet50_model=Resnet50Model()
resnet50_model.to(device)

num_epochs=10
resnet50_optimizer = torch.optim.AdamW(params=resnet50_model.parameters(),lr=0.001)
train_model(resnet50_model,dataloaders_dict,CustomLoss,resnet50_optimizer,num_epochs)